# Bài Tập Lớn Cuối Kì Môn Học Máy
* MSSV : 18020597
* Tên : Phạm Văn Hùng

# Giới thiệu và mô tả bài toán
Yêu cầu xây dựng thuật toán để đưa ra giá cả phù hợp cho từng sản phẩm. Ta sử dụng thông tin sản phẩm do Mercari cung cấp, được người bán mô tả các chi tiết của sản phẩm như tên thương hiệu sản phẩm, tên sản phẩm, tình trạng sản phẩm để dưa ra giá phù hợp nhất.

# Xử lý Dữ Liệu

### Khai báo các thư viện

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


### Giải nén các file nén dữ liệu

In [ ]:
!apt-get install p7zip
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/train.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/test.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/sample_submission.csv.7z
!unzip /kaggle/input/mercari-price-suggestion-challenge/sample_submission_stg2.csv.zip
!unzip /kaggle/input/mercari-price-suggestion-challenge/test_stg2.tsv.zip


### Dùng thư viện pandas để đọc dữ liệu từ bài toán vào dạng table

In [ ]:
train_dataset = pd.read_table("train.tsv")
test_dataset = pd.read_table("test_stg2.tsv")

# Phân tích dữ liệu

### Kiểm tra thông tin của tập dữ liệu train và test

In [ ]:
train_dataset.info()
test_dataset.info()

### Đưa ra số mẫu và số thuộc tính của từng dữ liệu

In [ ]:
print(train_dataset.shape)
print(test_dataset.shape)

* Tập train gồm 8 cột thông tin về sản phẩm và có tổng cộng 1482535 sản phẩm
* Tập test gồm 3460725 sản phẩm, có 7 cột thông tin và ta dự vào đó để tìm ra 'price' của sản phẩm

### Xuất ra thông tin về 5 sản phẩm đầu tiên của tập train

In [ ]:
train_dataset.head(5)

### Tổng quan về dữ liệu. 
Trong tập train ta có 8 trường thông tin dữ liệu: 
* train_id là chỉ số của sản phẩm 
* name id tên của sản phẩm (dạng text)
* item_condition_id là điều kiện của các mặt hàng này khi bán ra. (dạng số 1-5)
* category_name là danh mục của sản phẩm 
* brand_name là tên thương hiệu của sản phẩm 
* **price** là giá của sản phẩm và đây cũng là mục tiêu của bài toán này 
* shipping là sản phẩm này có được vận chuyển đến nhà (dạng số 0/1)
* item_description là mô tả thông tin sản phẩm (dạng văn bản)
> Do giá cả sản phẩm là mục tiêu của bài toán, nó là dạng dữ liệu số liên tục  
>> Bài toán này là dạng bài toán **Regression Linear Model**. 

### Lọc Dữ Liệu
* Ta loại bỏ các sản phẩm có price <= 0 
> Vì các sản phẩm giá <= 0 thì thường không có ý nghĩa cho mô hình

In [ ]:
train_dataset = train_dataset[train_dataset['price'] > 0].reset_index(drop = True)
# train, validation = train_test_split(train_dataset, test_size = 0.1, random_state = 30)
train = train_dataset.copy()
test = test_dataset.copy()
print(train.shape)
print(test.shape)

In [ ]:
unique_brands = train['brand_name'].value_counts()
print("Số nhãn hiệu phân biệt là {}".format(len(unique_brands)))
plt.figure(figsize=(15, 5))
sns.barplot(unique_brands.index[1:11], unique_brands[1:11])
plt.title('Top 10 các thương hiệu và số lượng sản phẩm của từng thương hiệu')
plt.xlabel('Tên thương hiệu')
plt.ylabel('Số lượng sản phẩm')
plt.show()

Nhận xét: 
* Nike và Victoria's Secret là 2 thương hiệu có số lượng sản phẩm đông nhất 
* Thương hiệu sản phẩm cũng một phần ảnh hưởng đến giá của sản phẩm đó

In [ ]:
count = train['item_condition_id'].value_counts()
plt.figure(figsize=(15, 5))
sns.barplot(count.index[:5], count[:5])
plt.ylabel('Số lượng sản phẩm')
plt.xlabel('Điều kiện sản phẩm')

In [ ]:
count = train['shipping'].value_counts()
plt.figure(figsize=(7, 3))
plt.subplot(1, 2, 1)
sns.barplot(count.index, count)
plt.xlabel('Shipping')
plt.ylabel('Số sản phẩm')
plt.subplot(1, 2, 2)
labels = ['0', '1']
sizes = count
colors = ['blue', 'orange']
explode = (0.1, 0)
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=140)
plt.axis('equal')
plt.show()

Nhận xét: 
* Phần lớn các sản phẩm đều được trả tiền vận chuyển bởi người mua. 

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

* Dựa vào quann sát ta thấy bên tập train có 672 ô category_name và 62797 ô brand_name bị null và bên test có khoảng 14833 giá trị null cho category_name và 1476490 giá trị null cho brand_name.  
> Ta nhận thấy cột category_name và cột brand_name có rất nhiều giá trị null. Vì thế ta cần thay thế các giá trị null bằng "Missing"

In [ ]:
def fill(data):
    data['category_name'].fillna('Missing', inplace = True)
    data['brand_name'].fillna('Missing', inplace = True)
    data['item_description'].fillna('Missing', inplace = True)
    return data

### Điền thêm vào các ô trống ở bảng train, test.

In [ ]:
fill(train)
fill(test)

Qua việc xuất ra thông tin của 5 sản phẩm đầu tiên của tập train và test, ta dễ nhận thấy cột category_name được chia thành 3 sub_category nhỏ và được chia cách nhau bởi kí tự '/'. 
> Vì vậy ta tách category thành 3 category nhỏ để kiểm tra thông tin dễ dàng hơn

In [ ]:
def split_category(cate):
    try:
        sub_cate1, sub_cate2, sub_cate3 = cate.split("/")
        return sub_cate1, sub_cate2, sub_cate3
    except:
        return ("Missing", "Missing", "Missing")
    
def creat_category(data):
    data['sub_category1'], data['sub_category2'], data['sub_category3'] = zip(*data['category_name'].apply(lambda x: split_category(x)))

### Tách trường dữ liệu "category_name" thành 3 thành phần dữ liệu con

In [ ]:
creat_category(train)
creat_category(test)

### Price (Giá của sản phẩm)

*Đây là mục tiêu dư đoán của bài toán vì vậy ta cần xem phân bố dữ liệu để biết được các tính chất của nó*

Ta kiểm tra phân phối của price(giá sản phẩm) trên tập train

In [ ]:
train.price.describe().apply(lambda x: format(x, 'f'))

Biểu diễn phân phối của biến price (gía cả) của các mặt hàng thông qua đồ thị

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.hist(train['price'], bins=50, range=[0, 200], edgecolor='white')
plt.title('Phân phối của giá sản phẩm')
plt.xlabel('Price(Giá)')
plt.ylabel('Số sản phẩm')
plt.subplot(1, 2, 2)
plt.hist(np.log1p(train['price']), bins=50, edgecolor='white')
plt.title('Phân phối của log(Price)')
plt.xlabel('log(Price)')
plt.ylabel('Số sản phẩm')
plt.show()
# sns.displot(np.log1p(train['price']))

# Trích Xuất Đặc Trưng Từ Dữ Liệu

### Áp dụng các kĩ thuật lấy đặc trưng dữ liệu
*      Dùng các phương pháp như : CountVectorizer, TfidfVectorizer, LabelBinarizer
*      Chuyển dữ liệu từ raw data mà máy tính chưa hiểu được sang các vector đặc trưng để các mô hình học máy có thể hiểu được

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from scipy.sparse import csr_matrix, hstack

In [ ]:
# Quy đổi thời gian 
import time
def format_time(seconds):
    days = int(seconds / 3600 / 24)
    seconds = seconds - days * 3600 * 24
    hours = int(seconds / 3600)
    seconds = seconds - hours * 3600
    minutes = int(seconds / 60)
    seconds = seconds - minutes * 60
    secondsf = int(seconds)
    seconds = seconds - secondsf
    millis = int(seconds * 1000)

    f = ''
    i = 1
    if days > 0:
        f += str(days) + 'D'
        i += 1
    if hours > 0 and i <= 2:
        f += str(hours) + 'h'
        i += 1
    if minutes > 0 and i <= 2:
        f += str(minutes) + 'm'
        i += 1
    if secondsf > 0 and i <= 2:
        f += str(secondsf) + 's'
        i += 1
    if millis > 0 and i <= 2:
        f += str(millis) + 'ms'
        i += 1
    if f == '':
        f = '0ms'
    return f

### Tạo vector đặc trưng cho "Name"

* Dùng phương pháp CountVectorizer để lấy đặc trưng cho trường NAME
* Phương pháp CountVectorizer sử dụng đếm tần suất các từ suất hiện trong một câu vào bảng từ vựng
![](https://www.educative.io/api/edpresso/shot/5197621598617600/image/6596233398321152)

In [ ]:
MAX_FEATURES_NAME = 20000
def name2vec(train_data, test_data):
    start_time = time.time()
    vectorizer = CountVectorizer(
        min_df=10,                         
        ngram_range=(1, 2),                 
        stop_words='english',
        max_features=MAX_FEATURES_NAME, 
    )
    train_vec = vectorizer.fit_transform(train_data)
    test_vec = vectorizer.transform(test_data)
    print('Name Completed: {}'.format(format_time(time.time() - start_time)))
    print('[Train] Name Shape: {}'.format(str(train_vec.shape)))
    print('[Test] Name Shape: {}'.format(str(test_vec.shape)))
    return train_vec, test_vec

In [ ]:
train_name, test_name = name2vec(train['name'], test['name'])

### Tạo vector đặc trưng cho "Item Condition" 

In [ ]:
def condition2vec(train_data, test_data):
    start_time = time.time()
    vectorizer = LabelBinarizer(sparse_output=True)
    train_vec = vectorizer.fit_transform(train_data)
    test_vec = vectorizer.transform(test_data)
    print('Condition Completed: {}'.format(format_time(time.time() - start_time)))
    print('[Train] Condition Shape: {}'.format(str(train_vec.shape)))
    print('[Test] Condition Shape: {}'.format(str(test_vec.shape)))
    return train_vec, test_vec

In [ ]:
train_condition, test_condition = condition2vec(train['item_condition_id'], test['item_condition_id'])

### Tạo vector đặc trưng cho "Shipping"

In [ ]:
def shipping2vec(train_data, test_data):
    start_time = time.time()
    vectorizer = LabelBinarizer(sparse_output=True)
    train_vec = vectorizer.fit_transform(train_data)
    test_vec = vectorizer.transform(test_data)
    print('Shipping Completed: {}'.format(format_time(time.time() - start_time)))
    print('[Train] Shipping Shape: {}'.format(str(train_vec.shape)))
    print('[Test] Shipping Shape: {}'.format(str(test_vec.shape)))
    return train_vec, test_vec

In [ ]:
train_ship, test_ship = shipping2vec(train['shipping'], test['shipping'])

### Tạo vector đặc trưng cho "Brand Name"

In [ ]:
def brandname2vec(train_data, test_data):
    start_time = time.time()
    vectorizer = LabelBinarizer(sparse_output=True)
    train_vec = vectorizer.fit_transform(train_data)
    test_vec = vectorizer.transform(test_data)
    print('BrandName Completed: {}'.format(format_time(time.time() - start_time)))
    print('[Train] BrandName Shape: {}'.format(str(train_vec.shape)))
    print('[Test] BrandName Shape: {}'.format(str(test_vec.shape)))
    return train_vec, test_vec

In [ ]:
train_bn, test_bn = brandname2vec(train['brand_name'], test['brand_name'])

### Tạo vector đặc trưng cho "Item Description"
* Vì dữ liệu là text nên ở đây ta dùng TFIDF để lấy đặc trưng cho dữ liệu
![](https://lh3.googleusercontent.com/proxy/zokXBvErIXGPXyQja4ayC-UKO5OLQnofcfLUfVRV-dPKWciw3VIvzmYhxIxvNkde0TPLt-lbhrQgWDcLIWzqqT_xDjnm8uD-huAYeCKFDJlO3Bd_CzvSg9huC-5_pDNRwZvaMqNk82c)

In [ ]:
MAX_FEATURES_ITEM_DESCRIPTION = 30000
def description2vec(train_data, test_data):
    start_time = time.time()
    vectorizer = TfidfVectorizer(
        max_features=MAX_FEATURES_ITEM_DESCRIPTION,
        ngram_range=(1, 2),
        stop_words='english'
    )
    train_vec = vectorizer.fit_transform(train_data)
    test_vec = vectorizer.transform(test_data)
    
    print('Description Completed: {}'.format(format_time(time.time() - start_time)))
    print('[Train] Description Shape: {}'.format(str(train_vec.shape)))
    print('[Test] Description Shape: {}'.format(str(test_vec.shape)))
    
    return train_vec, test_vec

In [ ]:
train_description, test_description = description2vec(train['item_description'], test['item_description']) 

### Tạo vector đặc trưng cho trường category

In [ ]:
start_time = time.time()
vectorizer = CountVectorizer()
train_sub1 = vectorizer.fit_transform(train['sub_category1'])
test_sub1 = vectorizer.transform(test['sub_category1'])

vectorizer = CountVectorizer()
train_sub2 = vectorizer.fit_transform(train['sub_category2'])
test_sub2 = vectorizer.transform(test['sub_category2'])

vectorizer = CountVectorizer()
train_sub3 = vectorizer.fit_transform(train['sub_category3'])
test_sub3 = vectorizer.transform(test['sub_category3'])

train_category = hstack([train_sub1, train_sub2, train_sub3])
test_category = hstack([test_sub1, test_sub2, test_sub3])
print('Category Completed: {}'.format(format_time(time.time() - start_time)))
print('[Train] Category Shape: {}'.format(str(train_category.shape)))
print('[Test] Category Shape: {}'.format(str(test_category.shape)))

### Gộp các vector đặc trưng đã lấy được ở từng mục thành ma trận tổng hợp cho tập train, test 
* Ở đây mỗi một sản phẩm sẽ được biểu diễn bởi một vector đặc trưng có số chiều là 55930. 

In [ ]:
train_X = hstack([train_name, train_bn, train_condition, train_category, train_description, train_ship]).tocsr()
test_X = hstack([test_name, test_bn, test_condition, test_category, test_description, test_ship]).tocsr()
print(train_X.shape)
print(test_X.shape)

### Tạo nhãn dữ liệu cho tập train

In [ ]:
train_Y = train_dataset["price"]

### Tách dữ liệu tập train ra thàn tập train và validation 
* Giữ lại 80% tập train là tâp dữ liệu huấn liệu
* Dùng 20% tập train là tập validation
> Dùng tập validation là tập dữ liệu không được huấn luyện để kiểm tra chất lượng của mô hình.
>> Phần này có thể giúp ta cho việc hyparameter tuning

In [ ]:
X_train, X_validation, y_train, y_validation = train_test_split(train_X, train_Y, test_size=0.1, random_state=42)

# Xây Dựng Các Mô Hình Học Máy Và Huấn Luyện Dữ Liệu

### Xây dựng các hàm đánh giá chất lượng mô hình


In [ ]:
def rmsle(y_pred, y):
    assert len(y) == len(y_pred)
    return np.sqrt(np.mean(np.power(np.log1p(y) - np.log1p(y_pred), 2)))

def evaluation(model):
    train_pred = model.predict(X_train)
    val_pred = model.predict(X_validation)
    train_score = rmsle(np.expm1(train_pred), y_train)
    validation_score = rmsle(np.expm1(val_pred), y_validation)
    print("Evaluation ---- Training Dataset Score: {}, Validation Dataset Score: {}".format(train_score, validation_score))
    return train_score, validation_score

def train(model):
    X, y = X_train, np.log1p(y_train)
    model.fit(X, y)
    evaluation(model)
    return model

### Khai báo các mô hình học máy

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

### Khởi Tạo mô hình và huấn luyện dữ liệu

### Ridge Regression

Ridge Regression về cơ bản cũng giống với Linear Regression truyền thống, vẫn sử dụng đến hàm mean square error nhưng có điểm khác là công thức tìm giá trị hệ số w thì bổ sung thêm ràng buộc để sao cho các hệ số w nhỏ nhất có thể đến mức gần bằng 0, nghĩa là các feature (x) ít có ảnh hưởng tới giá trị đầu ra. Ràng buộc này sử dụng công thức chuẩn hóa L2.

In [ ]:
ridge_model = Ridge(
    alpha=0.5, 
    copy_X=True, 
    fit_intercept=True, 
    max_iter=100,
    normalize=False, 
    random_state=101, 
    solver='auto', 
    tol=0.01
)
ridge_model = train(ridge_model)

### Lasso Regression
Một mô hình anh em với Ridge là Laso, thay vì Ridge sử dụng công thức chuẩn hóa L2 khi tìm hệ số w thì Laso sử dụng công thức chuẩn hóa L1. Sử dụng công thức chuẩn hóa L1 nhằm mục đích loại bớt feature ít ảnh hưởng tới đầu ra nghĩa là làm cho phần lớn các trọng số w = 0.

In [ ]:
lasso_model = Lasso(alpha=1e-06, fit_intercept=False)
lasso_model = train(lasso_model)

### LightGBM
LightGBM viết tắt của Light Gradient Boosting Machine, là mã nguồn mở xử lý thuật toán tăng cường độ dốc (Gradient Boosting) được phát triển bởi Microsoft. Gradient Boosting là một thuật toán xuất phát từ thuật toán Cây quyết định (Decision Tree), nó thực hiện việc xây dựng tuần tự nhiều Cây quyết định và tiến hành học tập. 

In [ ]:
import lightgbm as lgb
params = {
    'learning_rate': 0.75,
    'application': 'regression',
    'max_depth': 3,
    'num_leaves': 100,
    'verbosity': -1,
    'metric': 'RMSE',
}
d_train = lgb.Dataset(X_train, label=np.log1p(y_train))
d_valid = lgb.Dataset(X_validation, label=np.log1p(y_validation))
watchlist = [d_train, d_valid]

lgbm_model = lgb.train(
    params, 
    train_set=d_train, 
    num_boost_round=2200, 
    valid_sets=watchlist, 
    early_stopping_rounds=50, 
    verbose_eval=100
)

In [ ]:
from prettytable import PrettyTable
table = PrettyTable()
table.field_names = ["Model", "Train RMSLE", "Validation RMSLE"]
table.add_row(["Ridge Regression", 0.450944, 0.459817])
table.add_row(["Lasso Regression", 0.445953, 0.456407])
table.add_row(["LightGBM", 0.439119, 0.456571])
print(table)

### Nhận xét: 
* Các mô hình học máy Ridge Regression, Lasso Regression, LightGBM cũng không có nhiều sự khác biệt trên tập validation 
* Vì vậy các mô hình học máy không có tính chất ảnh hưởng nhiều đến kết quả bài toán, mà kết quả phần lớn bị ảnh hưởng nhất bởi phần trích xuất đặc trưng tạo ra vector đặc trưng cho từng sản phẩm. 
* Ngoài ra, ta cũng có thể áp dụng các kĩ thuật tìm kiếm để tìm kiếm bộ tham số tốt nhất cho các mô hình học máy

### Kết hợp các mô hình học máy
> Do các mô hình khác nhau nên sẽ học được các tính chất khác nhau của bài toán vì vậy ta kết hợp các mô hình lại để tạo ra được cái dự đoán có thể học được nhiều tính chất nhất có thể

In [ ]:
lasso_pred = lasso_model.predict(test_X)
ridge_pred = ridge_model.predict(test_X)
lgbm_pred = lgbm_model.predict(test_X)
test_pred = (lasso_pred + ridge_pred + lgbm_pred) / 3

### Chuẩn bị submission để nộp chấm điểm 

In [ ]:
submission: pd.DataFrame = test[['test_id']]
submission['price'] = np.expm1(test_pred)
submission.to_csv("submission.csv", index=False)

In [ ]:
submission.head()